# 0. Vorbereitung

In [166]:
import pandas as pd
import os
import glob


In [107]:
pd.set_option("display.max_columns", None)

Die nächste Zelle ist wichtig, denn hier werden die BA-Excel mit den Daten des jeweiligen Jahrgangs eingelesen, die dann im weiteren Verlauf umgeformt werden. Wenn mit anderen Jahrgängen gearbeitet werden soll, müssen Dateinamen, sheet names angepasst werden.

In [168]:

# Dateinamen und Tabellennamen
file_names = [
    ("230328_EA_338634_Gemeindedaten_Teil1.xlsx", "2014"),
    ("230328_EA_338634_Gemeindedaten_Teil3.xlsx", "2018"),
    ("230328_EA_338634_Gemeindedaten_Teil5.xlsx", "2021")
]

# Dictionary für die Dataframes
dataframes = {}

# Iteriere über die Dateien und Tabellennamen
for file_name, sheet_name in file_names:
    # Einlesen des spezifischen Tabellenblatts in ein DataFrame
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    # DataFrame im Dictionary speichern
    dataframes["ba_" + sheet_name] = df

# Zugriff auf die Dataframes
ba_2014 = dataframes["ba_2014"]
ba_2018 = dataframes["ba_2018"]
ba_2021 = dataframes["ba_2021"]

In [169]:
ba_2021.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Grundsicherung für Arbeitsuchende (SGB II)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Eckwerte der Grundsicherung nach Gemeinden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Deutschland - Gemeinden (Gebietsstand Dezember...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Jahresdurchschnitt 2021, Datenstand: März 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Daten zu Leistungen nach dem SGB II nach einer...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Gemeinde,Leistungs-berechtigte (LB),Insgesamt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,darunter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Männer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frauen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,dar. (Sp. 1) im Alter von ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dar. (Sp. 1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Leistungs-berechtigte (LB),dar. (Sp. 25) im Alter von ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dar. (Sp

# 1. BA-Daten umformen

### 1.1 Grundlegende Umformung

In [170]:
def transform_dataframe(df):
    # Zeilen 0 bis 7 entfernen
    df = df.drop(df.index[0:7])
    # Letzte Zeile entfernen
    df = df.drop(df.index[-1])
    
    # Spalte 'Unnamed: 0' nach Leerzeichen trennen und in neue Spalten 'AGS' und 'Gebiet' aufteilen
    df[['AGS', 'Gebiet']] = df['Unnamed: 0'].str.split(' ', 1, expand=True)
    # Spalte 'Unnamed: 0' entfernen
    df = df.drop(['Unnamed: 0'], axis=1)
    
    # Erstellen einer Kontrollspalte 'Gesamtwerte_Kontrolle' mit den Spalten 'AGS', 'Gebiet' und 'Unnamed: 1'
    Gesamtwerte_Kontrolle = df[['AGS', 'Gebiet', 'Unnamed: 1']].copy()
    
    # Spalte 'Unnamed: 1' entfernen
    df.drop(['Unnamed: 1'], axis=1, inplace=True)
    
    # Nur Spalten ab der 23. Spalte behalten
    df = df.iloc[:, 23:]
    
    # Fehlende Werte in Zeile 5 mit den Werten der vorherigen Zeilen füllen
    df.iloc[5] = df.iloc[5].fillna(df.iloc[4])
    df.iloc[5] = df.iloc[5].fillna(df.iloc[3])
    df.iloc[5] = df.iloc[5].fillna(df.iloc[2])
    
    # Zeilen 7, 9, 10, 11 und 13 entfernen
    df = df.drop([7, 9, 10, 11, 13])
    
    # Letzte beiden Werte in Zeile 1 mit den Werten der letzten beiden Spaltennamen füllen
    df.iloc[1, -1] = df.columns[-1]
    df.iloc[1, -2] = df.columns[-2]
    
    # Neue Spaltennamen aus Zeile 1 erstellen
    new_columns = df.iloc[1].tolist()
    # Spaltennamen aktualisieren
    df = df.set_axis(new_columns, axis=1, inplace=False)
    
    # Zeile 12 entfernen
    df = df.drop(12)
    
    # Transformiertes DataFrame zurückgeben
    return df


# Dataframes erstellen
ba_2014 = transform_dataframe(ba_2014)
ba_2018 = transform_dataframe(ba_2018)
ba_2021 = transform_dataframe(ba_2021)

In [172]:
ba_2018.head()

,Leistungs-berechtigte (LB),unter 65 Jahren,unter 15 Jahren,unter 3 Jahren,3 bis unter 6 Jahren,6 bis unter 15 Jahren,15 bis unter 18 Jahren,18 bis unter 25 Jahren,25 bis unter 65 Jahren,15 bis unter 18 Jahren (Kinder),Erwerbs-fähige Leistungs-berechtigte (ELB),erziehende ELB,allein-erziehende ELB,arbeit-suchende ELB insgesamt,kein Hauptschul-abschluss,Hauptschul-abschluss,mittlere Reife,Fachhoch-schule,Abitur/ Hochschul-reife,keine Angabe,ohne abge-schlossene Berufs-ausbildung,betriebliche/schulische Ausbildung,akade-mische Ausbildung,keine Angabe,Leistungs-berechtigte (LB),unter 65 Jahren,unter 15 Jahren,unter 3 Jahren,3 bis unter 6 Jahren,6 bis unter 15 Jahren,15 bis unter 18 Jahren,18 bis unter 25 Jahren,25 bis unter 65 Jahren,15 bis unter 18 Jahren (Kinder),Erwerbs-fähige Leistungs-berechtigte (ELB),erziehende ELB,allein-erziehende ELB,arbeit-suchende ELB insgesamt,kein Hauptschul-abschluss,Hauptschul-abschluss,mittlere Reife,Fachhoch-schule,Abitur/ Hochschul-reife,keine Angabe,ohne abge-schlossene Berufs-ausbildung,betriebliche/schulische Ausbildung,akade-mische Ausbildung,keine Angabe,AGS,Gebiet
8,Männer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frauen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,6128.67,6117.92,1561.5,378.75,341.167,841.583,229.75,741.667,3585,219.833,4499,949.25,101,3487.5,707.75,1321.83,457.583,210.417,388.75,401.167,2415.08,903.083,167.333,2,5643.08,5631.25,1571.83,396.25,330.083,845.5,208,676.667,3174.75,199.083,3974,1950.5,1077.67,2442.58,416.083,933.5,396.5,88.75,223.667,384.083,1628.67,724.333,87.25,2.33333,01001000,"Flensburg, Stadt"
15,17856,17793.5,4702.67,1010.42,985.417,2706.83,771.917,1870.08,10448.8,742.583,12939.1,3024.25,186.917,9929.67,2238.25,3303.83,1309.17,523.917,1707.92,846.583,6687.33,2449.67,789.083,3.58333,16340.4,16293.7,4562.42,993.833,930.833,2637.75,707.083,1609.75,9414.42,677.25,11555.5,5476.58,2516.08,7028.58,1520.17,2319.17,1166.75,315.667,1036.67,670.167,4652.67,1829.08,542.833,4,01002000,"Kiel, Landeshauptstadt"
16,13417.8,13344.7,3785.83,824.083,803.75,2158,599.583,1231.58,7727.67,579.583,9471.17,2197.92,163.25,7217.67,1469.5,2749.33,961.417,380.417,574.75,1082.25,4873.92,2011.75,329.333,2.66667,12890,12834.2,3357.92,693.25,717.667,1947,543.417,1218.5,7714.33,524.583,9347.25,4388.67,2277.58,5824.17,1027.83,2093.08,1005.92,252,491.417,953.917,3802.25,1756.75,263.333,1.83333,01003000,"Lübeck, Hansestadt"
17,4778.92,4760.5,1312.08,280.667,284.833,746.583,216.833,506.75,2724.83,210.25,3358.58,804,65,2581.92,605.75,1091.92,284,77.3333,147.667,375.25,1870.25,656.75,54.4167,0.5,4680.83,4658.58,1295.92,264.583,287.667,743.667,171,553.25,2638.42,160.5,3284.25,1630.33,874.667,2077.08,454.083,889.833,256.083,34.4167,98.25,344.417,1543.5,494.417,38.3333,0.833333,01004000,"Neumünster, Stadt"


### 1.2 Spalte für Geschlecht ergänzen und Tabelle entsprechend stapeln

In [115]:
def apply_transformation(df):
    # Extrahiere männliche Daten aus den ersten 24 Spalten und den Spalten 48 und 49
    df_m = df.iloc[:, list(range(24)) + [48, 49]]
    
    # Extrahiere weibliche Daten aus den Spalten 24 bis 49
    df_w = df.iloc[:, 24:50]
    
    # Setze das Geschlecht für die weiblichen Daten auf "w"
    df_w.loc[:, "Geschlecht"] = "w"
    
    # Setze das Geschlecht für die männlichen Daten auf "m"
    df_m.loc[:, "Geschlecht"] = "m"
    
    # Entferne Zeile 8 aus den weiblichen Daten
    df_w = df_w.drop([8])
    
    # Entferne Zeile 8 aus den männlichen Daten
    df_m = df_m.drop([8])
    
    # Kombiniere die männlichen und weiblichen Daten zu einem DataFrame
    combined_df = df_m.append(df_w, ignore_index=True)
    
    # Umbenenne Spalten, indem Bindestriche "-" entfernt werden
    combined_df = combined_df.rename(columns=lambda x: x.replace('-', ''))
    
    return combined_df

# Dataframes transformieren
ba_2014_combined = apply_transformation(ba_2014)
ba_2018_combined = apply_transformation(ba_2018)
ba_2021_combined = apply_transformation(ba_2021)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [173]:
ba_2014_combined.head()

,Leistungsberechtigte (LB),unter 65 Jahren,unter 15 Jahren,unter 3 Jahren,3 bis unter 6 Jahren,6 bis unter 15 Jahren,15 bis unter 18 Jahren,18 bis unter 25 Jahren,25 bis unter 65 Jahren,15 bis unter 18 Jahren (Kinder),Erwerbsfähige Leistungsberechtigte (ELB),erziehende ELB,alleinerziehende ELB,arbeitsuchende ELB insgesamt,kein Hauptschulabschluss,Hauptschulabschluss,mittlere Reife,Fachhochschule,Abitur/ Hochschulreife,keine Angabe,ohne abgeschlossene Berufsausbildung,betriebliche/schulische Ausbildung,akademische Ausbildung,keine Angabe,AGS,Gebiet,Geschlecht
0,5243.25,5236.42,1198.08,273.667,236.833,687.583,198.25,576.333,3263.75,193,3956.25,697.667,84.75,3215.67,564.083,1606.08,472,174.583,174,224.917,1984.5,1101.75,87.75,41.6667,01001000,"Flensburg, Stadt",m
1,16628.9,16599.8,4059.83,888.333,834.25,2337.25,684.75,1519.33,10335.8,670.167,12245.1,2532.17,177.167,9485.75,1735.08,4051.83,1469.75,509.5,1110.92,608.667,5776.25,3057.75,533.5,118.25,01002000,"Kiel, Landeshauptstadt",m
2,13740.6,13711.6,3487,745.167,707.583,2034.25,588.917,1278.92,8356.75,579.583,10053.9,2136.17,160.167,7943.33,1408.92,3825.75,1176.25,394.5,453.083,684.833,4916.42,2600.83,241.333,184.75,01003000,"Lübeck, Hansestadt",m
3,5154.42,5144.42,1287.08,268.833,241.25,777,242.5,501.333,3113.5,236.167,3761.92,843.333,78.5,2972.75,653.25,1499.5,339.917,71.5833,116.833,291.667,1914.17,963.917,41.75,52.9167,01004000,"Neumünster, Stadt",m
4,198,197,57.6667,17,7.16667,33.5,4.25,18.0833,117,4.25,136.5,27.5,1.16667,110.25,24.0833,60.25,13.0833,2.25,4.41667,6.16667,56.5833,51.5,1.83333,0.333333,01051001,Albersdorf,m


Prüfen: "arbeitsuchende ELB insgesamt" übernehmen, NICHT "Erwerbsfähige Leistungsberechtigte (ELB)" ... was wurde übernommen?

### 1.3 Gestapelte Zieltabellen erstellen:


 - Ein DF mit den LB nach Altersgruppen, das sich  auf Anzahl Leistungsberechtigte (LB) aggregieren lässt
     - Spalten "LB unter 15 Jahren" löschen, da Summe der Spalten 4-6 in der Excel
     - Spalten "LB unter 65 Jahren" löschen, da Summe der Spalten 4-9 in der Excel Untergruppen 
     - Spalte "15 bis unter 18 Jahren (Kinder)" löschen, da unklare Bedeutung...?
     - Spalte "Über 65 Jahren" als Differenz aus Spalte 1 in der Excel "Leistungsberechtigte (LB)" und Spalte 2 in der Excel 
     - "LB unter 65 Jahren" könnte man noch berechnen, aber nur sehr geringe Zahlen, daher zunächst unterlassen...
 - Ein DF mit den arbeitssuchenden ELB nach Schulabschlüssen
     - Spalten 15 - 20 in der Excel
 - Ein DF mit den arbeitssuchenden ELB nach Berufsausbildung
     - Spalten 21- 24 in der Excel
 - Ein DF mit den ELB nach Erziehungsstatus? Noch ausstehend, da unklar was benötigt wird
     - Spalten 11-13 in der Excel



In [174]:
def apply_transformation(df, year):
    # relevante Spalten aus der kombinierten Datenframe extrahieren
    LB_nach_Altersgruppen = df.iloc[:, [3, 4, 5, 6, 7, 8, 24, 25, 26]].copy()

    # Datensatz in das "lange" Format umwandeln, indem die Altersgruppen in eine separate Spalte gemeltet werden
    LB_nach_Altersgruppen = pd.melt(LB_nach_Altersgruppen, id_vars=['AGS', 'Gebiet', 'Geschlecht'], value_vars=['unter 3 Jahren', '3 bis unter 6 Jahren', '6 bis unter 15 Jahren', '15 bis unter 18 Jahren', '18 bis unter 25 Jahren', '25 bis unter 65 Jahren'], var_name='Altersgruppe', value_name='Anzahl')

    # Spalte hinzufügen, um das Jahr zu speichern
    LB_nach_Altersgruppen['Jahr'] = year

    # Spalten "AGS" und "Jahr" in Ganzzahlen umwandeln
    LB_nach_Altersgruppen["AGS"] = pd.to_numeric(LB_nach_Altersgruppen["AGS"], downcast="integer")
    LB_nach_Altersgruppen["Jahr"] = pd.to_numeric(LB_nach_Altersgruppen["Jahr"], downcast="integer")

    # gleicher Vorgang für die ELB arbeitssuchend nach Schulabschlüssen
    ELB_arbeitssuchend_nach_Schulabschlüssen = df.iloc[:, [14, 15, 16, 17, 18, 19, 24, 25, 26]].copy()
    ELB_arbeitssuchend_nach_Schulabschlüssen = pd.melt(ELB_arbeitssuchend_nach_Schulabschlüssen, id_vars=['AGS', 'Gebiet', 'Geschlecht'], value_vars=['kein Hauptschulabschluss', 'Hauptschulabschluss', 'mittlere Reife', 'Fachhochschule', 'Abitur/ Hochschulreife', 'keine Angabe'], var_name='Schulabschluss', value_name='Anzahl')
    ELB_arbeitssuchend_nach_Schulabschlüssen['Jahr'] = year
    ELB_arbeitssuchend_nach_Schulabschlüssen["AGS"] = pd.to_numeric(ELB_arbeitssuchend_nach_Schulabschlüssen["AGS"], downcast="integer")
    ELB_arbeitssuchend_nach_Schulabschlüssen["Jahr"] = pd.to_numeric(ELB_arbeitssuchend_nach_Schulabschlüssen["Jahr"], downcast="integer")

    # gleicher Vorgang für die ELB arbeitssuchend nach Berufsausbildung
    ELB_arbeitssuchend_nach_Berufsausbildung = df.iloc[:, [20, 21, 22, 23, 24, 25, 26]].copy()
    ELB_arbeitssuchend_nach_Berufsausbildung = pd.melt(ELB_arbeitssuchend_nach_Berufsausbildung, id_vars=['AGS', 'Gebiet', 'Geschlecht'], value_vars=['ohne abgeschlossene Berufsausbildung', 'betriebliche/schulische Ausbildung', 'akademische Ausbildung', 'keine Angabe'], var_name='Berufsausbildung', value_name='Anzahl')
    ELB_arbeitssuchend_nach_Berufsausbildung['Jahr'] = year
    ELB_arbeitssuchend_nach_Berufsausbildung["AGS"] = pd.to_numeric(ELB_arbeitssuchend_nach_Berufsausbildung["AGS"], downcast="integer")
    ELB_arbeitssuchend_nach_Berufsausbildung["Jahr"] = pd.to_numeric(ELB_arbeitssuchend_nach_Berufsausbildung["Jahr"], downcast="integer")

    # Rückgabe der Ergebnisse in separaten Dataframes
    return LB_nach_Altersgruppen, ELB_arbeitssuchend_nach_Schulabschlüssen, ELB_arbeitssuchend_nach_Berufsausbildung

# Funktion ausführen
LB_nach_Altersgruppen_2014, ELB_arbeitssuchend_nach_Schulabschlüssen_2014, ELB_arbeitssuchend_nach_Berufsausbildung_2014 = apply_transformation(ba_2014_combined, 2014)
LB_nach_Altersgruppen_2018, ELB_arbeitssuchend_nach_Schulabschlüssen_2018, ELB_arbeitssuchend_nach_Berufsausbildung_2018 = apply_transformation(ba_2018_combined, 2018)
LB_nach_Altersgruppen_2021, ELB_arbeitssuchend_nach_Schulabschlüssen_2021, ELB_arbeitssuchend_nach_Berufsausbildung_2021 = apply_transformation(ba_2021_combined, 2021)



In [186]:
ELB_arbeitssuchend_nach_Schulabschlüssen_2018.tail()

,AGS,Gebiet,Geschlecht,Schulabschluss,Anzahl,Jahr
132403,16077047,Thonhausen,w,keine Angabe,0.0833333,2018
132404,16077048,Treben,w,keine Angabe,0,2018
132405,16077049,Vollmershain,w,keine Angabe,0,2018
132406,16077051,Wildenbörten,w,keine Angabe,0.0833333,2018
132407,16077052,Windischleuba,w,keine Angabe,0,2018


# 2. Bevölkerungsdaten umformen



In [119]:
# Liste mit den Dateinamen erstellen
file_names = glob.glob('bevoelkerung_alter_geschlecht_*.csv')

# Schleife über alle Dateien
for file in file_names:
    # Jahreszahl aus dem Dateinamen extrahieren
    year = file.split('_')[-1].split('.')[0]

    # Daten einlesen und jede CSV in einem DataFrame mit der Bezeichnung "bevoelkerung_yyyy" speichern
    df_name = 'bevoelkerung_' + year
    globals()[df_name] = pd.read_csv(file, encoding='ISO-8859-1', delimiter=';')

    # Umwandlung der Spalte "Geschlecht"
    for df in [globals()[df_name]]:
        df['Geschlecht'] = df['Geschlecht'].astype(str)
        df['Geschlecht'] = df['Geschlecht'].replace({'2': 'w', '1': 'm'})

    # Umwandlung der Spalte "Alter"
    for df in [globals()[df_name]]:
        df['Alter'] = df['Alter'].replace({'u3': 'unter 3 Jahren', '3-u6': '3 bis unter 6 Jahren',
                                           '6-u15': '6 bis unter 15 Jahren', '15-u18': '15 bis unter 18 Jahren',
                                           '18-u25': '18 bis unter 25 Jahren', '25-u65': '25 bis unter 65 Jahren'})

    # Entfernung der Zeilen mit den Werten "u65" und "ü65" in der Spalte "Alter"
    for df in [globals()[df_name]]:
        df.drop(df[(df['Alter'] == 'u65') | (df['Alter'] == 'ü65') | (df['Alter'] == 'insg')].index, inplace=True)
    
    # Spaltennamen ändern      
    for df in [globals()[df_name]]:
        df.rename(columns={"Alter": "Altersgruppe", "Anzahl": "Anzahl_Bevoelkerung"}, inplace=True)


In [179]:
bevoelkerung_2014.head(10)

,AGS,Geschlecht,Altersgruppe,Anzahl_Bevoelkerung
1,1001000,m,unter 3 Jahren,1087.0
2,1001000,m,3 bis unter 6 Jahren,1054.0
3,1001000,m,6 bis unter 15 Jahren,3135.0
4,1001000,m,15 bis unter 18 Jahren,1163.0
5,1001000,m,18 bis unter 25 Jahren,4817.0
6,1001000,m,25 bis unter 65 Jahren,23068.0
10,1001000,w,unter 3 Jahren,1149.0
11,1001000,w,3 bis unter 6 Jahren,1013.0
12,1001000,w,6 bis unter 15 Jahren,2871.0
13,1001000,w,15 bis unter 18 Jahren,1117.0


#### Exkurs: herausfinden, wie sehr sich die Gebietskörperschaften der Jahrgänge unterscheiden

In [122]:
import itertools

# Liste der Dataframe-Namen
dfs = ["bevoelkerung_2014", "bevoelkerung_2015", "bevoelkerung_2016", "bevoelkerung_2017", "bevoelkerung_2018", "bevoelkerung_2019", "bevoelkerung_2020", "bevoelkerung_2021"]

# Leeres Dictionary, um die Ergebnisse zu speichern
results = {}

# Schleife über alle möglichen Paare von Dataframes
for pair in itertools.combinations(dfs, 2):
    df1 = pair[0]
    df2 = pair[1]
    
    # Vergleich der beiden Region-Spalten und Speicherung der Unterschiede in einem Set
    diff1 = set(eval(df1)['AGS']).difference(set(eval(df2)['AGS']))
    diff2 = set(eval(df2)['AGS']).difference(set(eval(df1)['AGS']))
    
    # Speicherung des Ergebnisses im Dictionary
    results[f"{df1} vs {df2}"] = {"diff1": diff1, "diff2": diff2}

# Ausgabe der Ergebnisse
for key, value in results.items():
    print(key)
    print("Anzahl der Unterschiede in " + key.split(" vs ")[0] + ":", len(value["diff1"]))
    print("Unterschiede in " + key.split(" vs ")[0] + ":", list(value["diff1"]))
    print("Anzahl der Unterschiede in " + key.split(" vs ")[1] + ":", len(value["diff2"]))
    print("Unterschiede in " + key.split(" vs ")[1] + ":", list(value["diff2"]))
    print("Gesamtanzahl der Unterschiede:", len(value["diff1"].union(value["diff2"])))
    print("-------------------------------------------------")

bevoelkerung_2014 vs bevoelkerung_2015
Anzahl der Unterschiede in bevoelkerung_2014: 29
Unterschiede in bevoelkerung_2014: [13071106, 3157003, 14521100, 3157004, 14729110, 3352003, 3352010, 3352012, 3352013, 3352015, 3352016, 3352027, 3352028, 3352030, 3352031, 3352034, 3352035, 3351013, 3352037, 3352040, 3153001, 3153004, 3351020, 3352047, 3352048, 3153011, 3153015, 3352057, 13071098]
Anzahl der Unterschiede in bevoelkerung_2015: 5
Unterschiede in bevoelkerung_2015: [3157009, 3351026, 3153018, 3352061, 3352062]
Gesamtanzahl der Unterschiede: 34
-------------------------------------------------
bevoelkerung_2014 vs bevoelkerung_2016
Anzahl der Unterschiede in bevoelkerung_2014: 111
Unterschiede in bevoelkerung_2014: [14522370, 3158024, 3157003, 3157004, 3158029, 3156002, 3156003, 3156006, 3156008, 3156009, 3156010, 3156011, 3156012, 3156013, 3156015, 3156016, 3156017, 3154011, 3154020, 3154023, 3153001, 3153004, 13076081, 3153011, 3153015, 3152001, 3152002, 3152003, 3152004, 3152005, 3

In [125]:
def check_value_in_dataframe(df_name, value):
    
    # Überprüft, ob der Wert in der Spalte 'AGS' des Dataframes enthalten ist:
    # True, wenn der Wert in der Spalte 'AGS' des Dataframes enthalten ist, sonst False.
    
    df = globals()[df_name]  # Zugriff auf den Dataframe über den Namen
    return value in df['AGS'].values

result = check_value_in_dataframe('bevoelkerung_2020', 16056000)
print(result)  # Ausgabe: True oder False, je nachdem ob der Wert in der Spalte 'AGS' enthalten ist...

True


# 3. Sozial- und Bevölkerungsdaten mergen und Quoten berechnen

## 3.1 Mergen

In [131]:
LB_nach_Altersgruppen_2021.head()

,AGS,Gebiet,Geschlecht,Altersgruppe,Anzahl,Jahr
0,1001000,"Flensburg, Stadt",m,unter 3 Jahren,359.333,2021
1,1002000,"Kiel, Landeshauptstadt",m,unter 3 Jahren,852.5,2021
2,1003000,"Lübeck, Hansestadt",m,unter 3 Jahren,587.583,2021
3,1004000,"Neumünster, Stadt",m,unter 3 Jahren,277.583,2021
4,1051001,Albersdorf,m,unter 3 Jahren,6.08333,2021


In [133]:
bevoelkerung_2021.head()

,AGS,Geschlecht,Altersgruppe,Anzahl_Bevoelkerung
1,1001000,m,unter 3 Jahren,1430.0
2,1001000,m,3 bis unter 6 Jahren,1330.0
3,1001000,m,6 bis unter 15 Jahren,3559.0
4,1001000,m,15 bis unter 18 Jahren,1149.0
5,1001000,m,18 bis unter 25 Jahren,4983.0


In [132]:
bevoelkerung_2021['Altersgruppe'].unique()

array(['unter 3 Jahren', '3 bis unter 6 Jahren', '6 bis unter 15 Jahren',
       '15 bis unter 18 Jahren', '18 bis unter 25 Jahren',
       '25 bis unter 65 Jahren'], dtype=object)

#### Plan fürs mergen (hier für 2021, aber analog auch auf die anderen jahrgänge übertragen):

#####    - LB_nach_Altersgruppen_2021 und bevoelkerung_2021: 
        - Ziel: Anteil der LB nach altersgruppen an der Gesamtzahl der LB u65 berechnen
        - Vorbereitung: bevoelkerung_2021 gruppieren nach "AGS" und "Geschlecht", 
        sodass Summe der u65 Jährigen gebildet wird für SGBII-Quote
        - merge über die Spalten "AGS", "Geschlecht"
#####    - ELB_arbeitssuchend_nach_Berufsausbildung_2021 und bevoelkerung 2021:
        - Ziel: Anteil der ELB nach Berufsausbildung und Geschlecht an der Gesamtzahl der ELB 15-u65 berechnen
        - Vorbereitung: bevoelkerung_2021 alle Zeilen droppen, wo "Altersgruppe" nicht "15 bis unter 18 Jahren",
        "18 bis unter 25 Jahren" oder "25 bis unter 65 Jahren" enthält, gruppieren nach "AGS" und "Geschlecht", 
        sodass Summe der 15-u65 Jährigen gebildet wird für ELB-Quote
        - merge über die Spalten "AGS", "Geschlecht"
#####    - ELB_arbeitssuchend_nach_Schulabschlüssen_2021 und bevoelkerung 2021:
        - Ziel: Anteil der ELB nach Schulabschluss und Geschlecht an der Gesamtzahl der ELB 15-u65 berechnen
        - Vorbereitung: bevoelkerung_2021 alle Zeilen droppen, wo "Altersgruppe" nicht "15 bis unter 18 Jahren", 
        "18 bis unter 25 Jahren" oder "25 bis unter 65 Jahren" enthält, gruppieren nach "AGS" und "Geschlecht", 
        sodass Summe der 15-u65 Jährigen gebildet wird für ELB-Quote
        - merge über die Spalten "AGS", "Geschlecht"

In [189]:
# dataframes für 2014 mergen

## LB

bevoelkerung_grouped = bevoelkerung_2014.groupby(["AGS", "Geschlecht"], as_index=False).agg({"Altersgruppe": "sum", "Anzahl_Bevoelkerung": "sum"})
merged_lb_bev_2014 = LB_nach_Altersgruppen_2014.merge(bevoelkerung_grouped, on=["AGS", "Geschlecht"])

## ELB nach Schulabschlüssen

bevoelkerung_filtered = bevoelkerung_2014[
    bevoelkerung_2014["Altersgruppe"].isin(["15 bis unter 18 Jahren", "18 bis unter 25 Jahren", "25 bis unter 65 Jahren"])
]
bevoelkerung_filtered_grouped = bevoelkerung_filtered.groupby(["AGS", "Geschlecht"], as_index=False).agg({"Altersgruppe": "sum", "Anzahl_Bevoelkerung": "sum"})
merged_elb_schulabschl_2014 = ELB_arbeitssuchend_nach_Schulabschlüssen_2014.merge(bevoelkerung_filtered_grouped, on=["AGS", "Geschlecht"])

## ELB nach Berufsausbildung

bevoelkerung_filtered = bevoelkerung_2014[
    bevoelkerung_2014["Altersgruppe"].isin(["15 bis unter 18 Jahren", "18 bis unter 25 Jahren", "25 bis unter 65 Jahren"])
]
bevoelkerung_filtered_grouped = bevoelkerung_filtered.groupby(["AGS", "Geschlecht"], as_index=False).agg({"Altersgruppe": "sum", "Anzahl_Bevoelkerung": "sum"})
merged_elb_berufsausb_2014 = ELB_arbeitssuchend_nach_Berufsausbildung_2014.merge(bevoelkerung_filtered_grouped, on=["AGS", "Geschlecht"])

# dataframes für 2018 mergen

## LB

bevoelkerung_grouped = bevoelkerung_2018.groupby(["AGS", "Geschlecht"], as_index=False).agg({"Altersgruppe": "sum", "Anzahl_Bevoelkerung": "sum"})
merged_lb_bev_2018 = LB_nach_Altersgruppen_2018.merge(bevoelkerung_grouped, on=["AGS", "Geschlecht"])

## ELB nach Schulabschlüssen

bevoelkerung_filtered = bevoelkerung_2018[
    bevoelkerung_2018["Altersgruppe"].isin(["15 bis unter 18 Jahren", "18 bis unter 25 Jahren", "25 bis unter 65 Jahren"])
]
bevoelkerung_filtered_grouped = bevoelkerung_filtered.groupby(["AGS", "Geschlecht"], as_index=False).agg({"Altersgruppe": "sum", "Anzahl_Bevoelkerung": "sum"})
merged_elb_schulabschl_2018 = ELB_arbeitssuchend_nach_Schulabschlüssen_2018.merge(bevoelkerung_filtered_grouped, on=["AGS", "Geschlecht"])

## ELB nach Berufsausbildung

bevoelkerung_filtered = bevoelkerung_2018[
    bevoelkerung_2018["Altersgruppe"].isin(["15 bis unter 18 Jahren", "18 bis unter 25 Jahren", "25 bis unter 65 Jahren"])
]
bevoelkerung_filtered_grouped = bevoelkerung_filtered.groupby(["AGS", "Geschlecht"], as_index=False).agg({"Altersgruppe": "sum", "Anzahl_Bevoelkerung": "sum"})
merged_elb_berufsausb_2018 = ELB_arbeitssuchend_nach_Berufsausbildung_2018.merge(bevoelkerung_filtered_grouped, on=["AGS", "Geschlecht"])

# dataframes für 2021 mergen

## LB

bevoelkerung_grouped = bevoelkerung_2021.groupby(["AGS", "Geschlecht"], as_index=False).agg({"Altersgruppe": "sum", "Anzahl_Bevoelkerung": "sum"})
merged_lb_bev_2021 = LB_nach_Altersgruppen_2021.merge(bevoelkerung_grouped, on=["AGS", "Geschlecht"])

## ELB nach Schulabschlüssen

bevoelkerung_filtered = bevoelkerung_2021[
    bevoelkerung_2021["Altersgruppe"].isin(["15 bis unter 18 Jahren", "18 bis unter 25 Jahren", "25 bis unter 65 Jahren"])
]
bevoelkerung_filtered_grouped = bevoelkerung_filtered.groupby(["AGS", "Geschlecht"], as_index=False).agg({"Altersgruppe": "sum", "Anzahl_Bevoelkerung": "sum"})
merged_elb_schulabschl_2021 = ELB_arbeitssuchend_nach_Schulabschlüssen_2021.merge(bevoelkerung_filtered_grouped, on=["AGS", "Geschlecht"])

## ELB nach Berufsausbildung

bevoelkerung_filtered = bevoelkerung_2021[
    bevoelkerung_2021["Altersgruppe"].isin(["15 bis unter 18 Jahren", "18 bis unter 25 Jahren", "25 bis unter 65 Jahren"])
]
bevoelkerung_filtered_grouped = bevoelkerung_filtered.groupby(["AGS", "Geschlecht"], as_index=False).agg({"Altersgruppe": "sum", "Anzahl_Bevoelkerung": "sum"})
merged_elb_berufsausb_2021 = ELB_arbeitssuchend_nach_Berufsausbildung_2021.merge(bevoelkerung_filtered_grouped, on=["AGS", "Geschlecht"])


In [190]:
merged_elb_schulabschl_2021.head()

,AGS,Gebiet,Geschlecht,Schulabschluss,Anzahl,Jahr,Altersgruppe,Anzahl_Bevoelkerung
0,1001000,"Flensburg, Stadt",m,kein Hauptschulabschluss,612.25,2021,15 bis unter 18 Jahren18 bis unter 25 Jahren25...,31016.0
1,1001000,"Flensburg, Stadt",m,Hauptschulabschluss,1186,2021,15 bis unter 18 Jahren18 bis unter 25 Jahren25...,31016.0
2,1001000,"Flensburg, Stadt",m,mittlere Reife,422.917,2021,15 bis unter 18 Jahren18 bis unter 25 Jahren25...,31016.0
3,1001000,"Flensburg, Stadt",m,Fachhochschule,181.25,2021,15 bis unter 18 Jahren18 bis unter 25 Jahren25...,31016.0
4,1001000,"Flensburg, Stadt",m,Abitur/ Hochschulreife,314.083,2021,15 bis unter 18 Jahren18 bis unter 25 Jahren25...,31016.0


### 3.2 Ergebnistabellen stapeln

In [191]:
stacked_lb_bev = pd.concat([merged_lb_bev_2014, merged_lb_bev_2018, merged_lb_bev_2021], ignore_index=True)

stacked_elb_schulabschl = pd.concat([merged_elb_schulabschl_2014, merged_elb_schulabschl_2018, merged_elb_schulabschl_2021], ignore_index=True)

stacked_elb_berufsausb = pd.concat([merged_elb_berufsausb_2014, merged_elb_berufsausb_2018, merged_elb_berufsausb_2021], ignore_index=True)

In [192]:
stacked_elb_schulabschl.tail()

,AGS,Gebiet,Geschlecht,Schulabschluss,Anzahl,Jahr,Altersgruppe,Anzahl_Bevoelkerung
392599,16077052,Windischleuba,w,Hauptschulabschluss,2.33333,2021,15 bis unter 18 Jahren18 bis unter 25 Jahren25...,540.0
392600,16077052,Windischleuba,w,mittlere Reife,4.33333,2021,15 bis unter 18 Jahren18 bis unter 25 Jahren25...,540.0
392601,16077052,Windischleuba,w,Fachhochschule,1,2021,15 bis unter 18 Jahren18 bis unter 25 Jahren25...,540.0
392602,16077052,Windischleuba,w,Abitur/ Hochschulreife,0,2021,15 bis unter 18 Jahren18 bis unter 25 Jahren25...,540.0
392603,16077052,Windischleuba,w,keine Angabe,0,2021,15 bis unter 18 Jahren18 bis unter 25 Jahren25...,540.0


In [193]:
stacked_elb_berufsausb["Altersgruppe"].unique()


array(['15 bis unter 18 Jahren18 bis unter 25 Jahren25 bis unter 65 Jahren'],
      dtype=object)

### 3.3 Ergebnistabellen Spalten umbennen

In [194]:
# Umbenennungen im DataFrame "stacked_lb_bev"
stacked_lb_bev = stacked_lb_bev.rename(columns={"Anzahl_Bevoelkerung": "Anzahl_Bevoelkerung_u65",
                                                "Altersgruppe_x": "Altersgruppe_LB",
                                                "Anzahl": "Anzahl_LB"})
stacked_lb_bev = stacked_lb_bev.drop(columns="Altersgruppe_y")

# Umbenennungen im DataFrame "stacked_elb_schulabschl"
stacked_elb_schulabschl = stacked_elb_schulabschl.rename(columns={"Anzahl_Bevoelkerung": "Anzahl_Bevoelkerung_15_bis_u65",
                                                                  "Anzahl": "Anzahl_ELB"})
stacked_elb_schulabschl = stacked_elb_schulabschl.drop(columns="Altersgruppe")

# Umbenennungen im DataFrame "stacked_elb_berufsausb"
stacked_elb_berufsausb = stacked_elb_berufsausb.rename(columns={"Anzahl_Bevoelkerung": "Anzahl_Bevoelkerung_15_bis_u65",
                                                                "Anzahl": "Anzahl_ELB"})
stacked_elb_berufsausb = stacked_elb_berufsausb.drop(columns="Altersgruppe")

In [195]:
stacked_elb_berufsausb.head()

,AGS,Gebiet,Geschlecht,Berufsausbildung,Anzahl_ELB,Jahr,Anzahl_Bevoelkerung_15_bis_u65
0,1001000,"Flensburg, Stadt",m,ohne abgeschlossene Berufsausbildung,1984.5,2014,29048.0
1,1001000,"Flensburg, Stadt",m,betriebliche/schulische Ausbildung,1101.75,2014,29048.0
2,1001000,"Flensburg, Stadt",m,akademische Ausbildung,87.75,2014,29048.0
3,1001000,"Flensburg, Stadt",m,keine Angabe,41.6667,2014,29048.0
4,1002000,"Kiel, Landeshauptstadt",m,ohne abgeschlossene Berufsausbildung,5776.25,2014,84000.0


In [208]:
# wie hoch ist anteil der gemeinden mit unter 100 Einwohnern
weniger_100 = stacked_lb_bev['Anzahl_Bevoelkerung_u65'][stacked_lb_bev['Anzahl_Bevoelkerung_u65'] < 100].count()

insgesamt = stacked_lb_bev['Anzahl_Bevoelkerung_u65'].count()

anteil = (weniger_100 / insgesamt) * 100 

print(f"Anteil der AGS mit weniger als 100 an Gesamtzahl de r AGS: {anteil}%")

Anteil der AGS mit weniger als 100 an Gesamtzahl de r AGS: 8.972399669896385%


### 3.3 Quoten berechnen 

In [197]:
# Berechnung der SGB2-Quote (%)

stacked_lb_bev["SGB2-Quote (%)"] = (stacked_lb_bev["Anzahl_LB"] / stacked_lb_bev["Anzahl_Bevoelkerung_u65"]) * 100
stacked_lb_bev.tail()

,AGS,Gebiet,Geschlecht,Altersgruppe_LB,Anzahl_LB,Jahr,Anzahl_Bevoelkerung_u65,SGB2-Quote (%)
392599,16077052,Windischleuba,w,3 bis unter 6 Jahren,0,2021,664.0,0
392600,16077052,Windischleuba,w,6 bis unter 15 Jahren,0,2021,664.0,0
392601,16077052,Windischleuba,w,15 bis unter 18 Jahren,0,2021,664.0,0
392602,16077052,Windischleuba,w,18 bis unter 25 Jahren,0.5,2021,664.0,0.0753012
392603,16077052,Windischleuba,w,25 bis unter 65 Jahren,7.91667,2021,664.0,1.19227


In [198]:
# Berechnung der ELB-Quote nach Berufsausbildung (%)
stacked_elb_berufsausb["ELB-Quote (%)"] = (stacked_elb_berufsausb["Anzahl_ELB"] / stacked_elb_berufsausb["Anzahl_Bevoelkerung_15_bis_u65"]) * 100
stacked_elb_berufsausb.head()

,AGS,Gebiet,Geschlecht,Berufsausbildung,Anzahl_ELB,Jahr,Anzahl_Bevoelkerung_15_bis_u65,ELB-Quote (%)
0,1001000,"Flensburg, Stadt",m,ohne abgeschlossene Berufsausbildung,1984.5,2014,29048.0,6.8318
1,1001000,"Flensburg, Stadt",m,betriebliche/schulische Ausbildung,1101.75,2014,29048.0,3.79286
2,1001000,"Flensburg, Stadt",m,akademische Ausbildung,87.75,2014,29048.0,0.302086
3,1001000,"Flensburg, Stadt",m,keine Angabe,41.6667,2014,29048.0,0.143441
4,1002000,"Kiel, Landeshauptstadt",m,ohne abgeschlossene Berufsausbildung,5776.25,2014,84000.0,6.87649


In [199]:
# Berechnung der ELB-Quote nach Schulabschlus (%)
stacked_elb_schulabschl["ELB-Quote (%)"] = (stacked_elb_schulabschl["Anzahl_ELB"] / stacked_elb_schulabschl["Anzahl_Bevoelkerung_15_bis_u65"]) * 100
stacked_elb_schulabschl.tail()

,AGS,Gebiet,Geschlecht,Schulabschluss,Anzahl_ELB,Jahr,Anzahl_Bevoelkerung_15_bis_u65,ELB-Quote (%)
392599,16077052,Windischleuba,w,Hauptschulabschluss,2.33333,2021,540.0,0.432099
392600,16077052,Windischleuba,w,mittlere Reife,4.33333,2021,540.0,0.802469
392601,16077052,Windischleuba,w,Fachhochschule,1,2021,540.0,0.185185
392602,16077052,Windischleuba,w,Abitur/ Hochschulreife,0,2021,540.0,0
392603,16077052,Windischleuba,w,keine Angabe,0,2021,540.0,0


### 3.4 Tabellen exportieren als Excel und CSV

In [105]:
# DataFrame stacked_elb_schulabschl speichern
stacked_elb_schulabschl.to_excel('elb_quote_schulabschl_14_18_21.xlsx', index=False)
stacked_elb_schulabschl.to_csv('elb_quote_schulabschl_14_18_21.csv', index=False)

# DataFrame stacked_elb_berufsausb speichern
stacked_elb_berufsausb.to_excel('elb_quote_berufsausb_14_18_21.xlsx', index=False)
stacked_elb_berufsausb.to_csv('elb_quote_berufsausb_14_18_21.csv', index=False)

# DataFrame stacked_lb_bev speichern
stacked_lb_bev.to_excel('sgb2_quote_14_18_21.xlsx', index=False)
stacked_lb_bev.to_csv('sgb2_quote_14_18_21.csv', index=False)

In [153]:
stacked_elb_schulabschl["ELB-Quote (%)"].describe()

count     392604.0
unique    163509.0
top            0.0
freq      114626.0
Name: ELB-Quote (%), dtype: float64

In [156]:

# Höchste Werte
topten = stacked_elb_schulabschl.sort_values('ELB-Quote (%)', ascending=False).head(10)[['Gebiet', 'Jahr', 'ELB-Quote (%)']]

print("\nTop 10:")
print(topten)


Top 10:
                    Gebiet  Jahr ELB-Quote (%)
157567                 Ueß  2018       19.4444
61            Bergewöhrden  2014       16.6667
138823           Freistatt  2018       15.7676
118861          Voigtsdorf  2014       15.1961
55579          Groß Luckow  2014       14.7135
268885              Büttel  2021       14.2857
270973           Freistatt  2021       13.9501
119132          Dalkendorf  2014       13.5684
149603         Heinzenberg  2018       13.0952
118208  Demmin, Hansestadt  2014       12.8546
